In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
zz

In [2]:
act_fn = nn.ReLU(inplace=True)
def conv(ni, nf, ks=3, stride=1, bias=False):
    return nn.Conv1d(ni, nf, kernel_size=ks, stride=stride, padding=ks//2, bias=bias)

In [3]:
def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
    bn = nn.BatchNorm1d(nf)
    nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
    layers = [conv(ni, nf, ks, stride=stride), bn]
    if act: layers.append(act_fn)
    return nn.Sequential(*layers)

In [54]:
class InceptionModule(nn.Module):
    def __init__(self, ni, use_bottleneck=True, kss=[41, 21, 11], bottleneck_size=32, nb_filters=32, stride=1):
        super().__init__()
        if use_bottleneck:
            self.conv0 = nn.Conv1d(ni, bottleneck_size, 1, bias=False)
        else:
            self.conv0 = noop
        self.conv1 = conv(nb_filters, nb_filters, kss[0])
        self.conv2 = conv(nb_filters, nb_filters, kss[1])
        self.conv3 = conv(nb_filters, nb_filters, kss[2])   
        self.conv_bottle = nn.Sequential(nn.MaxPool1d(3, stride, padding=1), 
                                         nn.Conv1d(nb_filters, nb_filters, 1, bias=False))
        self.bn_relu = nn.Sequential(nn.BatchNorm1d(4*nb_filters), 
                                     nn.ReLU())
    def forward(self, x):
        x = self.conv0(x)
        return torch.cat([self.conv1(x), self.conv2(x), self.conv3(x), self.conv_bottle(x)], dim=1)

In [3]:
im = InceptionModule(1).cuda(); im

NameError: name 'InceptionModule' is not defined

In [ ]:
x = torch.rand(16, 1, 100).cuda()

In [57]:
im(x).shape

torch.Size([16, 128, 100])

In [58]:
im2 = InceptionModule(128).cuda()

In [59]:
im(x).shape

torch.Size([16, 128, 100])

In [67]:
MergeLayer??

In [62]:
model = SequentialEx(InceptionModule(1),
                      MergeLayer(),
                      InceptionModule(128), 
                      MergeLayer(),
                      InceptionModule(128)
                      MergeLayer(), 
                      )

In [15]:
inception_module(1, True, 41, 32, 32)

SequentialEx(
  (layers): ModuleList(
    (0): Conv1d(1, 32, kernel_size=(1,), stride=(1,), bias=False)
    (1): Conv1d(32, 32, kernel_size=(41,), stride=(1,), padding=(20,), bias=False)
    (2): Conv1d(32, 32, kernel_size=(20,), stride=(1,), padding=(10,), bias=False)
    (3): Conv1d(32, 32, kernel_size=(10,), stride=(1,), padding=(5,), bias=False)
    (4): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (5): Conv1d(32, 32, kernel_size=(1,), stride=(1,), bias=False)
  )
)

In [66]:
model.cuda()(x).shape

torch.Size([16, 128, 100])

In [ ]:
class Classifier_INCEPTION(nn.Module):
    def __init__(self, input_shape, nb_classes,nb_filters=32, use_residual=True, 
                 use_bottleneck=True, depth=6, kernel_size=41):
        super().__init__()
        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.batch_size = batch_size
        self.bottleneck_size = 32
        
        for i in range(depth):
            
        
        
        def forward(self, x):
            input_res = x
            for d in range(self.depth):

                x = self._inception_module(x)

                if self.use_residual and d % 3 == 2:
                    x = self._shortcut_layer(input_res, x)
                    input_res = x

In [ ]:
kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]